In [ ]:
import pandas as pd
from O365 import *
import os
import hvac

In [ ]:
# self.__secrets = self.read_secret(self.__destination["TYPE"], self.__destination["VARS"])

@staticmethod
def read_secret(mount_point, path):
    client = hvac.Client(url=os.environ.get("VAULT_ADDR"))
    resp = client.kv.v2.read_secret(mount_point=mount_point, path=f'/{path}')
    secret = resp['data']['data']

    return secret


In [ ]:
secrets = read_secret("azuread", "app_01")

In [ ]:
scopes = [
    "Mail.Read.Shared",
    "Mail.ReadWrite.Shared",
    "Mail.Send.Shared",
]

In [ ]:
from configparser import SectionProxy
from azure.identity import DeviceCodeCredential
from kiota_authentication_azure.azure_identity_authentication_provider import (
    AzureIdentityAuthenticationProvider)
from msgraph import GraphRequestAdapter, GraphServiceClient
from msgraph.generated.me.me_request_builder import MeRequestBuilder
from msgraph.generated.me.mail_folders.item.messages.messages_request_builder import (
    MessagesRequestBuilder)
from msgraph.generated.me.send_mail.send_mail_post_request_body import SendMailPostRequestBody
from msgraph.generated.models.message import Message
from msgraph.generated.models.item_body import ItemBody
from msgraph.generated.models.body_type import BodyType
from msgraph.generated.models.recipient import Recipient
from msgraph.generated.models.email_address import EmailAddress


device_code_credential = DeviceCodeCredential(
    client_id=secrets["client_id"],
    tenant_id=secrets["tenant_id"],
)

auth_provider = AzureIdentityAuthenticationProvider(
    device_code_credential,
    scopes=scopes)
adapter = GraphRequestAdapter(auth_provider)
client = GraphServiceClient(adapter)

In [ ]:
users = client.users

In [ ]:
async def get_users():
    users = client.users
    users = await users.request().get()
    return users

In [ ]:
results = get_users()

In [ ]:
for user in results:
    print(user)

In [ ]:
messages.close()

In [ ]:
user_client.me.registered_devices.graph_endpoint.get()

In [ ]:
query_params = MessagesRequestBuilder.MessagesRequestBuilderGetQueryParameters(
    # Only request specific properties
    select=['from', 'isRead', 'receivedDateTime', 'subject'],
    # Get at most 25 results
    top=25,
    # Sort by received time, newest first
    orderby=['receivedDateTime DESC']
)
request_config = MessagesRequestBuilder.MessagesRequestBuilderGetRequestConfiguration(
    query_parameters= query_params
)

messages = await user_client.me.mail_folders.by_mail_folder_id('inbox').messages.get(
    request_configuration=request_config)
